In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime
from collections import defaultdict

In [3]:
df = pd.read_csv('Netflix_rating_reduced.csv')

In [4]:
df_train, df_test = train_test_split(df, stratify=df['User_ID'], test_size=0.3, random_state=13)

In [8]:
df_train.shape

(2304765, 3)

In [55]:
df_test.shape

(987757, 3)

In [12]:
trainset = df_train.pivot_table(df_train, index='User_ID', columns='Movie_ID')
trainset.fillna(0., inplace=True)

In [13]:
testset = df_train.pivot_table(df_train, index='User_ID', columns='Movie_ID')
testset.fillna(0., inplace=True)

In [17]:
train = trainset.to_numpy()
test = testset.to_numpy()

In [22]:
test.shape

(27329, 1350)

In [83]:
kk = []
for u in range(train.shape[0]):
    for i in range(train.shape[1]):
        if train[u,i] != 0:
            kk.append(len(np.intersect1d(R_u[u], S_k_r[i])))

NameError: name 'R_u' is not defined

In [95]:
class integrated_model():
    def __init__(self, train, test, factors, neighbors, epochs, verbose = False):
        self.R = train
        self.test = test
        self.n_users, self.n_items = train.shape
        self.gamma1 = 0.007
        self.gamma2 = 0.007       #learning_rate
        self.gamma3 = 0.001                    #learning_rate 
        self.lambda6 = 0.005                   #regularization_parameter
        self.lambda7 = 0.015 
        self.lambda8 = 0.015     #regularization_parameter
        self.f = factors                       #factor_size
        self.k = neighbors
        self.epochs = epochs
        self.verbose = verbose
        
        np.random.seed(101)
        self.P = np.random.standard_normal((self.n_users, self.f))
        self.Q = np.random.standard_normal((self.n_items, self.f))
        self.y = np.random.standard_normal((self.n_items, self.f))
        self.w = np.random.standard_normal((self.n_items, self.n_items))
        self.c = np.random.standard_normal((self.n_items, self.n_items))
        self.imp = (train > 0).astype(float)
        (self.R_u, self.S_k_r), (self.N_u, self.S_k_n) = self.get_neighbor(train, self.imp)
        self.R_k_iu = []
        self.N_k_iu = []
        #biases term
        self.b_u = np.zeros(self.n_users)
        self.b_i = np.zeros(self.n_items)
        self.mu = np.mean(self.R[np.where(self.R != 0)])
        self.squared_error = defaultdict(float)
    
    def get_neighbor(self, train, imp, lambda2=100):
        neighbor = []
        imp = self.imp
        for data in [train, imp]:
            rho = np.corrcoef(data.T)
            rho = np.nan_to_num(rho, nan = -1)
            n_ij = np.dot(imp.T, imp)
            
            S = n_ij/(n_ij+lambda2) * rho
            np.fill_diagonal(S, -1)
            R_u = {u: data[u,:].nonzero() for u in range(len(data))}
            S_k = {i: np.argsort(S[i,:])[-self.k] for i in range(len(S))}
            neighbor.append((R_u, S_k))
        return neighbor
    
    def fit(self):
        start = datetime.now()
        for epoch in range(self.epochs):
            n=0
            for u in range(self.n_users):
                N_u = np.where(self.imp[u,:] != 0)[0]
                for i in range(self.n_items):
                    if self.R[u,i] > 0:
                        R_k_iu = np.intersect1d(self.R_u[u], self.S_k_r[i])
                        N_k_iu = np.intersect1d(self.N_u[u], self.S_k_n[i])
                        self.squared_error[epoch] += self.gradient_descent(u, i, N_u, R_k_iu, N_k_iu)
                        n+=1
            self.squared_error[epoch] = np.sqrt(self.squared_error[epoch]/n)
            
            #if ([epoch+1] % 10 == 0) and verbose == True:
             #   print(f'epoch {epoch+1}: Training RMSE {self.loss[epoch]:.4f}')
        end = datatime.now()
        print(f'Training times {end-start}')
    
    def prediction(self, u, i, N_u, R_k_iu, N_k_iu):
        p = self.P[u] + np.sum(self.y[N_u], axis = 0)/np.sqrt(len(N_u))
        factor = np.dot(p, self.Q[i].T)
        neighbor_exp = 0
        neighbor_imp = 0
        if len(R_k_iu) > 0:
            bias_diff = self.R[u,R_k_iu] - (self.mu + self.b_u[u] + self.b_i[R_k_iu])
        else:
            bias_diff = 0
            neighbor_exp = 0
        if len(N_k_iu) > 0:
            neighbor_imp = np.sum(self.c[i,N_k_iu])/np.sqrt(len(N_k_iu))
        else:
            neighbor_imp = 0
        return self.mu + self.b_u[u] + self.b_i[i] + factor + neighbor_exp + neighbor_imp , bias_diff
    
    def gradient(self, u, i, N_u, R_k_iu, N_k_iu):
        pred, bias_weight = self.prediction(u, i, N_u, R_k_iu, N_k_iu)
        error = self.R[u,i] - pred
        
        dbu = error - self.lambda6 * self.b_u[u]
        dbi = error - self.lambda6 * self.b_i[i]
        dq = error * (self.P[u] + (np.sum(self.y[N_u], axis = 0)/np.sqrt(len(N_u)))) - self.lambda7 * self.Q[i]
        dp = error * self.Q[i] - self.lambda7 * self.P[u]
        dy = (error * self.Q[i]/np.sqrt(len(self.N_u))).reshape(1, -1) - self.lambda7 * self.y[N_u]
        dw = error * bias_weight/np.sqrt(len(self.R_k_iu)) - self.lambda8 * self.w[i,R_k_iu]
        dc = error/np.sqrt(len(N_k_iu)) - self.lambda8 * self.c[i, N_k_iu]
        return dbu, dbi, dq, dp, dy, dw, dc, error**2
    
    def gradient_descent(self, u, i, N_u, R_k_iu, N_k_iu):
        dbu, dbi, dq, dp, dy, dw, dc, squared_error = self.gradient(u, i, N_u, R_k_iu, N_k_iu)
        self.b_u[u] += self.gamma1 * dbu
        self.b_i[i] += self.gamma1 * dbi
        self.Q[i] += self.gamma2 * dq
        self.P[u] += self.gamma2 * dp
        self.y[N_u] += self.gamma2 * dy
        if len(R_k_iu) > 0:
            self.w[i,R_k_iu] += self.gamma3 * dw
        if len(N_k_iu) > 0:
            self.c[i,N_k_iu] += self.gamma3 * dc
        return squared_error

In [96]:
model = integrated_model(train, test, 10, 10, 10, verbose = True)

In [97]:
model.fit()

C:\Users\ahn\AppData\Local\Temp/ipykernel_15912/2981196525.py:92: RuntimeWarning: invalid value encountered in double_scalars
  dw = error * bias_weight/np.sqrt(len(self.R_k_iu)) - self.lambda8 * self.w[i,R_k_iu]
C:\Users\ahn\AppData\Local\Temp/ipykernel_15912/2981196525.py:93: RuntimeWarning: divide by zero encountered in double_scalars
  dc = error/np.sqrt(len(N_k_iu)) - self.lambda8 * self.c[i, N_k_iu]
C:\Users\ahn\AppData\Local\Temp/ipykernel_15912/2981196525.py:92: RuntimeWarning: divide by zero encountered in true_divide
  dw = error * bias_weight/np.sqrt(len(self.R_k_iu)) - self.lambda8 * self.w[i,R_k_iu]
C:\Users\ahn\AppData\Local\Temp/ipykernel_15912/2981196525.py:92: RuntimeWarning: invalid value encountered in subtract
  dw = error * bias_weight/np.sqrt(len(self.R_k_iu)) - self.lambda8 * self.w[i,R_k_iu]
C:\Users\ahn\AppData\Local\Temp/ipykernel_15912/2981196525.py:104: RuntimeWarning: invalid value encountered in add
  self.w[i,R_k_iu] += self.gamma3 * dw


KeyboardInterrupt: 

In [82]:
integrated_model.R_u

AttributeError: type object 'integrated_model' has no attribute 'R_u'

In [30]:
imp = (train > 0).astype(float)
for data in [train,imp]:
    rho = np.corrcoef(data.T)
    rho = np.nan_to_num(rho, nan = -1)
    n_ij = 

In [36]:
help(np.nan_to_num)

Help on function nan_to_num in module numpy:

nan_to_num(x, copy=True, nan=0.0, posinf=None, neginf=None)
    Replace NaN with zero and infinity with large finite numbers (default
    behaviour) or with the numbers defined by the user using the `nan`, 
    `posinf` and/or `neginf` keywords.
    
    If `x` is inexact, NaN is replaced by zero or by the user defined value in
    `nan` keyword, infinity is replaced by the largest finite floating point 
    values representable by ``x.dtype`` or by the user defined value in 
    `posinf` keyword and -infinity is replaced by the most negative finite 
    floating point values representable by ``x.dtype`` or by the user defined 
    value in `neginf` keyword.
    
    For complex dtypes, the above is applied to each of the real and
    imaginary components of `x` separately.
    
    If `x` is not inexact, then no replacements are made.
    
    Parameters
    ----------
    x : scalar or array_like
        Input data.
    copy : bool, optio